<a href="https://colab.research.google.com/github/ajou-y0uC4N/y0uC4N/blob/main/%EB%B9%85%EC%9D%91%EB%B3%B4_case5_AE_IF_LOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [210]:

data = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type5.csv')#.to_numpy()


X = data[data['type'] == 'Normal']
y = X['type']
y = y.map({'Normal': 0, 'Attack': 1}).to_numpy()
X = X.drop(columns =['type'])
X['messageID'] = X['messageID'].astype('category')
X = X.drop(columns =['isCan0','isService','serviceNODEID'])
X = X.drop(columns = ['time','data[0]','data[7]'])
print(X.shape)


(129996, 11)


In [225]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
input_dim = X_scaled.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Autoencoder 학습
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/30
3250/3250 [==============================] - 9s 3ms/step - loss: 0.6162 - val_loss: 0.5670
Epoch 2/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5772 - val_loss: 0.5649
Epoch 3/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5761 - val_loss: 0.5644
Epoch 4/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5760 - val_loss: 0.5640
Epoch 5/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5759 - val_loss: 0.5637
Epoch 6/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5759 - val_loss: 0.5636
Epoch 7/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5758 - val_loss: 0.5636
Epoch 8/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5758 - val_loss: 0.5635
Epoch 9/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.5758 - val_loss: 0.5635
Epoch 10/30
3250/3250 [==============================] - 8s 2ms/step - loss: 0.575

In [226]:
# 데이터를 인코딩
X_encoded = autoencoder.predict(X_scaled)
print(X_encoded.shape)

4063/4063 [==============================] - 5s 1ms/step
(129996, 11)


In [227]:
#data.loc[data['type']=='Normal', 'type'] = 0
#data.loc[data['type']=='Attack', 'type'] = 1

fullDataX1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type5.csv')#.to_numpy()

#X['messageID'] = data['messageID']
fX1 = fullDataX1.drop(columns =['type'])
fX1['messageID'] = fX1['messageID'].astype('category')
fX1 = fX1.drop(columns =['isCan0','isService','serviceNODEID'])
fX1 = fX1.drop(columns = ['time','data[0]','data[7]'])
print(fX1.shape)

#fX1_encoded = encoder_model.predict(fX1_scaled)
#print(fX1_encoded.shape)

(180608, 11)


In [228]:
fX1_scaled = scaler.fit_transform(fX1)
fX1_encoded = autoencoder.predict(fX1_scaled)

5644/5644 [==============================] - 7s 1ms/step


In [229]:
if_model = IsolationForest(contamination=0.15)  # contamination 값 조정 가능
if_model.fit(X_encoded)

# 이상치 예측
if_predictions = if_model.predict(fX1_encoded)
fy = fullDataX1['type']
fy = fy.map({'Normal': 1, 'Attack': 0}).to_numpy()
if_predictions = np.where(if_predictions == -1, 0, if_predictions)
print(np.unique(if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy, if_predictions)
class_report = classification_report(fy, if_predictions)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([103271,  77337]))
Confusion Matrix:
[[45587  5025]
 [57684 72312]]

Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.90      0.59     50612
           1       0.94      0.56      0.70    129996

    accuracy                           0.65    180608
   macro avg       0.69      0.73      0.65    180608
weighted avg       0.80      0.65      0.67    180608



In [40]:
from sklearn.model_selection import GridSearchCV
if_model = IsolationForest(contamination=0.2)  # contamination 값 조정 가능

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_samples': ['auto', 0.5, 0.75],
    'contamination': [0.05, 0.1, 0.2],
    'max_features': [1.0, 0.5, 0.75],
    'bootstrap': [False, True]
}

# GridSearchCV 사용
grid_search = GridSearchCV(estimator=if_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_scaled, y)

# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", grid_search.best_params_)

# 이상치 예측


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found:  {'bootstrap': False, 'contamination': 0.05, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 50}


In [41]:
best_params = grid_search.best_params_
best_if_model = IsolationForest(**best_params,)
best_if_model.fit(X_scaled)

import joblib
joblib.dump(best_if_model, './best_if_model.pkl')

# 모델 로드 및 novelty=True로 설정
if_model = best_if_model# joblib.load('./best_if_model.pkl')
#if_model.set_params(novelty=True)


if_predictions = if_model.predict(fX1_encoded)

In [42]:
from sklearn.metrics import confusion_matrix, classification_report
fy1 = fullDataX1['type']
fy1 = fy1.map({'Normal': 0, 'Attack': 1}).to_numpy()
if_predictions = np.where(if_predictions == -1, 0, if_predictions)
print(np.unique(if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy1, if_predictions)
class_report = classification_report(fy1, if_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)
print(np.unique(if_predictions, return_counts=True))

(array([0, 1]), array([    64, 180544]))
Confusion Matrix:
[[    64 129932]
 [     0  50612]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00    129996
           1       0.28      1.00      0.44     50612

    accuracy                           0.28    180608
   macro avg       0.64      0.50      0.22    180608
weighted avg       0.80      0.28      0.12    180608

(array([0, 1]), array([    64, 180544]))


In [43]:
# 모델 저장
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_if_model.pkl', 'wb') as file:
#    pickle.dump(if_model, file)

In [44]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_if_model.pkl', 'rb') as file:
    loaded_if_model = pickle.load(file)

# 불러온 모델로 예측
fy1 = fullDataX1['type']
fy1 = fy1.map({'Normal': 0, 'Attack': 1}).to_numpy()
loaded_if_predictions = loaded_if_model.predict(fX1_encoded)
loaded_if_predictions = np.where(loaded_if_predictions == -1, 1, loaded_if_predictions)
print(np.unique(loaded_if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy1, loaded_if_predictions)
class_report = classification_report(fy1, loaded_if_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([1]), array([180608]))
Confusion Matrix:
[[     0 129996]
 [     0  50612]]

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00    129996
           1       0.28      1.00      0.44     50612

    accuracy                           0.28    180608
   macro avg       0.14      0.50      0.22    180608
weighted avg       0.08      0.28      0.12    180608



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
# Local Outlier Factor 모델 정의 및 학습
lof_model = LocalOutlierFactor(n_neighbors=30, contamination=0.1, novelty=True)  # n_neighbors 및 contamination 값 조정 가능
lof_model.fit(X_encoded)
lof_predictions = lof_model.predict(fX1_encoded)
# 이상치라면 -1, 정상치라면 1로 표시됨


In [46]:
# 모델 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case5_lof_model.pkl', 'wb') as file:
    pickle.dump(lof_model, file)

In [47]:
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case5_lof_model.pkl', 'rb') as file:
    loaded_lof_model = pickle.load(file)

# 불러온 모델로 예측
loaded_lof_predictions = loaded_lof_model.predict(fX1_encoded)
print(np.unique(loaded_lof_predictions, return_counts=True))

(array([-1,  1]), array([143599,  37009]))


In [48]:
print(np.unique(lof_predictions, return_counts=True))

(array([-1,  1]), array([143599,  37009]))
